In [57]:
# Old Model

import torch
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import sqlite3
import pandas as pd

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.dropout1 = nn.Dropout(0.3)
        self.fc2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(64)
        self.dropout2 = nn.Dropout(0.3)
        self.fc3 = nn.Linear(32, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.dropout1(x)
        x = torch.relu(self.fc2(x))
        x = self.dropout2(x)
        x = self.fc3(x)
        return self.softmax(x)

conn = sqlite3.connect('../test_mTSP.sqlite3')
test_instances = pd.read_sql_query("SELECT * FROM instances", conn)
test_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)

conn.close()

test_features = test_instances[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
scaler = StandardScaler()
test_features = scaler.fit_transform(test_features)

x_test = torch.tensor(test_features, dtype=torch.float32)

with open('labels_order.txt') as f:
    label_order = [line.strip() for line in f]
    
# model
input_size = x_test.shape[1]
output_size = len(label_order)
model = NeuralNetwork(input_size, output_size)
model.load_state_dict(torch.load('models/best_model.pth'))
model.eval()


with torch.no_grad():
    predictions = model(x_test)
    predicted_classes = torch.argmax(predictions, dim=1)

predicted_algorithms = [label_order[class_idx.item()] for class_idx in predicted_classes]
test_instances['predicted_algorithm'] = predicted_algorithms

# calculate composite score for each real algorithm
test_algorithms = test_algorithms.merge(
    test_instances[['instance_id', 'nr_cities']],
    on='instance_id',
    how='left'
)

# Normalize time_taken și distance_gap global 
conn = sqlite3.connect('../train_mTSP.sqlite3')
train_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

time_min = train_algorithms['time_taken'].min()
time_max = train_algorithms['time_taken'].max()
gap_min = train_algorithms['distance_gap'].min()
gap_max = train_algorithms['distance_gap'].max()

test_algorithms['time_taken_norm'] = (test_algorithms['time_taken'] - time_min) / (time_max - time_min + 1e-8)
test_algorithms['distance_gap_norm'] = (test_algorithms['distance_gap'] - gap_min) / (gap_max - gap_min + 1e-8)

# Penalize branch and cut on large instances
penalty = (
    (test_algorithms['strategy'].str.lower() == 'branch and cut') &
    (test_algorithms['nr_cities'] > 40)
).astype(float) * (0.1)


test_algorithms['composite_score'] = test_algorithms['normalized_cost'] * 0.65 + \
                                     test_algorithms['time_taken_norm'] * 0.25 + \
                                     test_algorithms['distance_gap_norm'] * 0.1 + penalty 

best_real = test_algorithms.loc[test_algorithms.groupby('instance_id')['composite_score'].idxmin()]
best_real = best_real[['instance_id', 'strategy']].rename(columns={'strategy': 'real_algorithm'})


# compare predictions with real best algorithms
comparison = test_instances[['instance_id', 'predicted_algorithm']].merge(best_real, on='instance_id')
comparison['match'] = comparison['predicted_algorithm'] == comparison['real_algorithm']

final_comparison = comparison[['instance_id', 'predicted_algorithm', 'real_algorithm', 'match']]
final_comparison.to_csv('comparatie_pred_vs_real_version1.csv', index=False)

accuracy = final_comparison['match'].mean()
print(f"Acuratetea retelei neuronale noi: {accuracy:.2%}")
print(final_comparison.head())


Acuratetea retelei neuronale noi: 65.35%
   instance_id predicted_algorithm  real_algorithm  match
0            1      Branch and Cut  Branch and Cut   True
1            2      Branch and Cut  Branch and Cut   True
2            3      Branch and Cut  Branch and Cut   True
3            4      Branch and Cut  Branch and Cut   True
4            5      Branch and Cut  Branch and Cut   True


In [69]:
# New Model

import torch
import torch.nn as nn
import sqlite3
import pickle
import pandas as pd
import numpy as np

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            nn.Linear(32, output_size)
        )
    def forward(self, x):
        return self.net(x)

# Load test data
conn = sqlite3.connect('../test_mTSP.sqlite3')
test_instances = pd.read_sql_query("SELECT * FROM instances", conn)
test_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

# Extract test features
test_features = test_instances[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]

# Load scaler
with open('scaler_version2.pkl', 'rb') as f:
    scaler = pickle.load(f)
test_features_scaled = scaler.transform(test_features)
x_test = torch.tensor(test_features_scaled, dtype=torch.float32)

# Load label order
with open('labels_order-v2.txt') as f:
    label_order = [line.strip() for line in f]

# Load model
input_size = x_test.shape[1]
output_size = len(label_order)
model = NeuralNetwork(input_size, output_size)
model.load_state_dict(torch.load('models_version2/best_model.pth'))
model.eval()

# Predict
with torch.no_grad():
    predictions = model(x_test)

    predicted_classes = torch.argmax(predictions, dim=1)
    predicted_algorithms = [label_order[class_idx.item()] for class_idx in predicted_classes]

test_instances['predicted_algorithm'] = predicted_algorithms

test_algorithms = test_algorithms.merge(
    test_instances[['instance_id', 'nr_cities']],
    on='instance_id',
    how='left'
)

# Load train stats for normalization
conn = sqlite3.connect('../train_mTSP.sqlite3')
train_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

time_min = train_algorithms['time_taken'].min()
time_max = train_algorithms['time_taken'].max()
gap_min = train_algorithms['distance_gap'].min()
gap_max = train_algorithms['distance_gap'].max()

# Normalize
test_algorithms['time_taken_norm'] = (test_algorithms['time_taken'] - time_min) / (time_max - time_min + 1e-8)
test_algorithms['distance_gap_norm'] = (test_algorithms['distance_gap'] - gap_min) / (gap_max - gap_min + 1e-8)

# Penalize branch and cut on large instances
penalty = (
    (test_algorithms['strategy'].str.lower() == 'branch and cut') &
    (test_algorithms['nr_cities'] > 40)
).astype(float) * (0.1)

test_algorithms['composite_score'] = test_algorithms['normalized_cost'] * 0.65 + \
                                     test_algorithms['time_taken_norm'] * 0.25 + \
                                     test_algorithms['distance_gap_norm'] * 0.1 + penalty 

# Find best real algorithm per instance
best_real = test_algorithms.loc[test_algorithms.groupby('instance_id')['composite_score'].idxmin()]
best_real = best_real[['instance_id', 'strategy']].rename(columns={'strategy': 'real_algorithm'})

# Compare predictions with real best
comparison = test_instances[['instance_id', 'predicted_algorithm']].merge(best_real, on='instance_id')
comparison['match'] = comparison['predicted_algorithm'] == comparison['real_algorithm']

# Save results
final_comparison = comparison[['instance_id', 'predicted_algorithm', 'real_algorithm', 'match']]
final_comparison.to_csv('comparatie_pred_vs_real_version2.csv', index=False)

accuracy = final_comparison['match'].mean()
print(f"Acuratetea retelei neuronale noi: {accuracy:.2%}")
print(final_comparison.head())  


Acuratetea retelei neuronale noi: 86.14%
   instance_id predicted_algorithm  real_algorithm  match
0            1      Branch and Cut  Branch and Cut   True
1            2      Branch and Cut  Branch and Cut   True
2            3      Branch and Cut  Branch and Cut   True
3            4      Branch and Cut  Branch and Cut   True
4            5      Branch and Cut  Branch and Cut   True
